## Bootstrap

In [337]:
import numpy as np 
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Дана выборка из генеральной совокупности и необходимо оценить выбоочное среднее $\theta$. Сформируем $N$ подвыборок, которые будут формироваться случайным выбором с возвращением, выбранные элементы исходной выборки возвращаются в выборку и далее могут быть использованы снова.

Формально на каждом шаге мы выбираем жлемент исходной выборки с вероятностью $\frac{1}{n}$, поэтому из генеральной совокупности будут *не выбраны*:
$$\lim\limits_{n\ \to \infty} \left(1-\frac{1}{n}\right)^n = e^{-1} \approx 0.36$$

Для построения доверительного интервала для среднего значения будем использовать аналог статистики для нормального распределения:
$$\sqrt{n}\cdot \frac{\bar{X}-\mu}{\sigma} \sim \sqrt{n}\cdot \frac{\bar{X}^*-\bar{X}}{S_n}$$
где

- $\bar{X}^*$ - среднее значение по bootstrap выборке
- $\bar{X}$ - среднее значение по исходной
- $S_n$ - среднеквадратичное отклонение (выборочное, если дисперсия неизвестна начальная)

Тогда доверительный интервал можно построить двумя способами:

- обычный доверительный интервал через квантиль нормального распределения 
$$ \bar{X} - q_{\frac{\alpha}{2}}\cdot \frac{S_n}{\sqrt{n}} \leq \bar{X}^* \leq \bar{X} + q_{\frac{1 - \alpha}{2}}\cdot \frac{S_n}{\sqrt{n}}$$

- просто от bootstrap средних отсечь хвосты распределений и получить 95% интервал

В таблице `result` первые две колонки - нижняя и верхняя граница соответствующего доверительного интервала по первому способу, а 3-4 - соответственно по второму.

## Functions

In [338]:
def sample_bootstrap(data, n_samples):
    '''
    функция реализует возвращение подвыборок из генеральной совокупности
    
    ----
    
    data - исходные данные
    n_samples - количество разбиений 
    
    output: индексы элементов
    '''
    n = len(data)
    idx = np.random.randint(low = 0, high = n, size = (n_samples, n))
    samples = data[idx]
    return samples

def stat_intervals(stat,alpha=0.05):
    '''
    функция возвращает доверительный интервал вторым способом
    
    alpha - доверительный уровень
    
    stat - массив статистик (средних в нашем случае)
    
    '''
    bound = np.percentile(sorted(stat),[100*alpha/2.,100*(1-alpha/2.)])
    
    return bound

### Preprocessing

In [339]:
directory = 'data/*.txt' 
files = glob.glob(directory)

for i, file in enumerate(files):
    exec(f"x_{i} = pd.read_csv(files[i], header=None, sep = '\t',names = {[str(file)]}, decimal=',') ")

## Results

In [340]:
np.random.seed(0)
alpha = 0.05
result = pd.DataFrame(columns = [f'{alpha/2}_left_normal',f'{1-alpha/2}_right_normal',f'{alpha/2}_left_percentile',f'{1-alpha/2}_right_percentile'])
a = np.array([])

for i,x in enumerate([x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8]):
    
    true_mean = np.mean(x.values) # среднее исходных данных
    
    true_std = np.std(x.values) # стандартное отклонение исходных данных
    
    x_0_mean_score = np.array(list(map(np.mean, sample_bootstrap(x.values,1000)))) #bootstrap выборки
    
    mean = np.mean(x_0_mean_score) #среднее по boostratrp средним

    res = stat_intervals(x_0_mean_score,alpha) #доверительный интервал 2-ой споосб
    
    left = mean - scipy.stats.norm.ppf(1-alpha/2)*true_std/np.sqrt(x.shape[0]) #левая граница 1-ого способа
    right = mean + scipy.stats.norm.ppf(1-alpha/2)*true_std/np.sqrt(x.shape[0]) #правая граница 2-ого способа
    
    result.loc[x.columns[0]]=np.array([left,right,res[0],res[1]])

In [343]:
result = result.round(3)
result

,0.025_left_normal,0.975_right_normal,0.025_left_percentile,0.975_right_percentile
data\10_uM_STL.txt,105.954,277.057,118.823,283.192
data\200_uM_ITP.txt,38.593,206.662,54.329,220.152
data\200_uM_ITP_10_uM_STL.txt,150.135,444.235,161.335,454.739
data\400_uM_NTP.txt,19.816,103.517,31.462,109.440
data\400_uM_NTP_10_uM_STL.txt,132.376,571.116,168.601,601.356
data\5_uM_GreA_10_uM_STL.txt,123.911,313.650,128.241,312.554
data\5_uM_GreA_D41A.txt,63.868,482.184,90.505,504.585
data\5_uM_GreA_D41A_10_uM_STL.txt,217.761,401.894,227.898,406.469
data\5_uM_GreB_10_uM_STL.txt,277.035,503.143,285.408,517.281
